In [1]:
import           Control.Monad.Trans.State.Strict
import qualified Data.IntMap.Strict  as M
import qualified Data.Vector         as V

import           Control.Monad (forM_, when)
import           Data.Maybe (isNothing, fromJust)
import           Prelude hiding (lookup)

In [2]:
graph = M.fromList
    [ (1, [2])
    , (2, [3])
    , (3, [1])
    , (4, [2, 3, 5])
    , (5, [4, 6])
    , (6, [7, 3])
    , (7, [6])
    , (8, [5, 7, 8])
    ]

In [3]:
data TState = TState
    { index    :: Int
    , stack    :: [Int]
    , stackSet :: V.Vector Bool
    , stackTop :: Maybe Int
    , indices  :: V.Vector (Maybe Int)
    , lowlinks :: V.Vector (Maybe Int)
    , output   :: [[Int]]
    } deriving (Eq, Show)
  
data Event
    = IncrementIndex
    | InsertIndex   Int Int
    | InsertLowlink Int Int
    | Push Int
    | Pop
    | OutputSCC [Int]
    deriving (Eq, Show)

data EState = EState
    { current :: TState
    , history :: [Event]
    } deriving (Eq, Show)

insert :: V.Vector a -> Int -> a -> V.Vector a
insert vector index value = vector V.// [(index, value)]

apply :: Event -> TState -> TState
apply event state = case event of
    IncrementIndex    -> state { index    = index state + 1}
    InsertIndex k v   -> state { indices  = insert (indices state) k (Just v) }
    InsertLowlink k v -> state { lowlinks = insert (lowlinks state) k (Just v) }
    Push i            -> state
        { stack    = i : (stack state) 
        , stackSet = insert (stackSet state) i True
        }
    Pop               -> let i = head (stack state) in state
        { stack    = tail (stack state)
        , stackSet = insert (stackSet state) i False
        , stackTop = Just i
        }
    OutputSCC scc     -> state { output = output state ++ [scc]}

initial = TState 0 [] newVector' Nothing newVector newVector []
     where 
        size = (fst $ M.findMax graph) + 1
        newVector  = V.replicate size Nothing :: V.Vector (Maybe Int)
        newVector' = V.replicate size False

emit :: Event -> State EState TState
emit event = do
    modify' $ \s -> s
        { current = apply event (current s) 
        , history = event : (history s)
        }
    gets current

In [4]:
tarjan :: M.IntMap [Int] -> ([[Int]], EState)
tarjan graph = flip runState (EState initial []) $ do
    forM_ (M.keys graph) $ \v -> do
        is <- indices <$> gets current
        when (isNothing (is V.! v)) $ strongConnect v graph
    output <$> gets current

strongConnect :: Int -> M.IntMap [Int] -> State EState ()
strongConnect v graph = do
    emit IncrementIndex
    i <- index <$> gets current
    emit $ InsertIndex   v i
    emit $ InsertLowlink v i
    emit $ Push v
    forM_ (graph M.! v) $ \w -> do
        is <- indices <$> gets current
        case is V.! w of
            Nothing -> do
                strongConnect w graph
                ls <- lowlinks <$> gets current
                let vLowlink = fromJust (ls V.! v)
                let wLowlink = fromJust (ls V.! w)
                emit $ InsertLowlink v (min vLowlink wLowlink)
                pure ()
            Just wIndex -> do
                sSet <- stackSet <$> gets current
                when (sSet V.! w) $ do
                    ls <- lowlinks <$> gets current
                    let vLowlink = fromJust (ls V.! v)
                    emit $ InsertLowlink v (min vLowlink wIndex)
                    pure ()
    s <- gets current
    let vLowlink = fromJust (lowlinks s V.! v)
    let vIndex   = fromJust (indices s V.! v)
    when (vLowlink == vIndex) $ do
        scc <- addSCC v []
        emit $ OutputSCC scc
        pure ()

addSCC :: Int -> [Int] -> State EState [Int]
addSCC v scc = do
    emit Pop
    w <- fromJust . stackTop <$> gets current
    let scc' = w:scc
    if w == v
        then return scc'
        else addSCC v scc'

In [5]:
tarjan graph

([[1,2,3],[6,7],[4,5],[8]],EState {current = TState {index = 8, stack = [], stackSet = [False,False,False,False,False,False,False,False,False], stackTop = Just 8, indices = [Nothing,Just 1,Just 2,Just 3,Just 4,Just 5,Just 6,Just 7,Just 8], lowlinks = [Nothing,Just 1,Just 1,Just 1,Just 4,Just 4,Just 6,Just 6,Just 8], output = [[1,2,3],[6,7],[4,5],[8]]}, history = [OutputSCC [8],Pop,InsertLowlink 8 8,Push 8,InsertLowlink 8 8,InsertIndex 8 8,IncrementIndex,OutputSCC [4,5],Pop,Pop,InsertLowlink 4 4,InsertLowlink 5 4,OutputSCC [6,7],Pop,Pop,InsertLowlink 6 6,InsertLowlink 7 6,Push 7,InsertLowlink 7 7,InsertIndex 7 7,IncrementIndex,Push 6,InsertLowlink 6 6,InsertIndex 6 6,IncrementIndex,InsertLowlink 5 4,Push 5,InsertLowlink 5 5,InsertIndex 5 5,IncrementIndex,Push 4,InsertLowlink 4 4,InsertIndex 4 4,IncrementIndex,OutputSCC [1,2,3],Pop,Pop,Pop,InsertLowlink 1 1,InsertLowlink 2 1,InsertLowlink 3 1,Push 3,InsertLowlink 3 3,InsertIndex 3 3,IncrementIndex,Push 2,InsertLowlink 2 2,InsertIndex 2 2